In [25]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Assuming 'df' is your DataFrame
pd.set_option('display.max_rows', 1000)  # Show all rows
pd.set_option('display.max_columns', 1000)  # Show all columns
pd.set_option('display.width', None)  # Adjust width to avoid truncation
pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [26]:
api = wandb.Api()

In [27]:
entity = 'timlachner' 
project = 'pricing_cMDP_test'
run_ids = ["840qghy3", "2q3512op", "22pwd968", "p0j6rpjl"]
run_names = ["Clairvoyant", "SAC", "Greedy", "ILQX"]

In [28]:
runs = []
for run_id in run_ids:
    runs.append(api.run(f"{entity}/{project}/{run_id}"))

In [44]:
# Initialize an empty list to store dataframes
dfs = []

# Loop through each run and get the dataframe
for run, name in zip(runs, run_names):
    df = run.history()
    df['Run'] = name
    dfs.append(df)

# Concatenate all dataframes
merged_df = pd.concat(dfs, ignore_index=True)
# Get the True_Cumulative_Reward of the Clairvoyant run
clairvoyant_rewards = merged_df[merged_df['Run'] == 'Clairvoyant']['True_Cumulative_Reward'].reset_index(drop=True)

# Calculate the True_Regret for each run
merged_df['True_Regret'] = merged_df.apply(lambda row: clairvoyant_rewards[row.name % len(clairvoyant_rewards)]-row['True_Cumulative_Reward'], axis=1)


In [46]:
merged_df

,Epoch,_step,t,True_Cumulative_Reward,_runtime,Cumulative_Reward,Action,Inventory,True_Reward,Reward,_timestamp,Run,True_Regret
0,0,0,0,13.272383,135.160332,14.348712,2.83,30000,13.272383,14.348712,1.742286e+09,Clairvoyant,0.000000
1,0,1,1,20.310029,135.160801,21.479380,1.75,30000,7.037646,7.130668,1.742286e+09,Clairvoyant,0.000000
2,0,2,2,36.555836,135.160814,36.436696,3.56,30000,16.245807,14.957316,1.742286e+09,Clairvoyant,0.000000
3,0,3,3,48.974325,135.161011,49.593352,2.70,30000,12.418489,13.156656,1.742286e+09,Clairvoyant,0.000000
4,0,4,4,58.870419,135.161194,59.523157,2.32,30000,9.896094,9.929804,1.742286e+09,Clairvoyant,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,9995,9995,136117.152742,106.461767,135958.294707,3.19,30000,12.538437,14.103192,1.742286e+09,ILQX,2493.333649
39996,0,9996,9996,136125.655130,106.461780,135966.832074,2.18,30000,8.502388,8.537367,1.742286e+09,ILQX,2493.395383
39997,0,9997,9997,136146.980395,106.461795,135991.161135,5.16,30000,21.325265,24.329061,1.742286e+09,ILQX,2493.558570
39998,0,9998,9998,136160.272124,106.461810,136003.146193,3.28,30000,13.291729,11.985058,1.742286e+09,ILQX,2493.659677


In [55]:
from lets_plot import *

LetsPlot.setup_html()

# Plot the last value of True_Cumulative_Reward for each Run
last_values = merged_df.groupby('Run').tail(1)[['Run', 'True_Regret']]
last_values = last_values[last_values['Run'] != 'Clairvoyant']
# Create the boxplot
p = ggplot(last_values, aes(x='Run', y='True_Regret')) + geom_boxplot()
p.show()

In [52]:
from lets_plot import *

LetsPlot.setup_html()

# Plot the last value of True_Cumulative_Reward for each Run
last_values = merged_df.groupby('Run').tail(1)[['Run', 'True_Cumulative_Reward']]

# Create the boxplot
p = ggplot(last_values, aes(x='Run', y='True_Cumulative_Reward')) + geom_boxplot()
p.show()